# Libs

In [1]:
from psw import MyClass # access information
import openai # openai lib
import tiktoken # lib used by openai the tokenize words
import pandas as pd

# Main Variables

In [2]:
access = MyClass()
openai.api_key = access.key # your key
model = access.model['model']['name'] # or 'gpt-3.5-turbo-0613'
max_tokens = access.model['model']['limit_tokens'] # or 4096
input_cost = access.model['model']['input'] / 1000 # OpenIA charges by 1k tokens, dividing by 1000, will bring the cost by token
output_cost = access.model['model']['output'] / 1000 # same thing

# Functions

In [3]:
def input_text():
    text = input("user: ")
    return text

In [4]:
def input_size(model, cost_per_token, text):
    encoding = tiktoken.encoding_for_model(model)
    text_encoded = encoding.encode(text)
    number_tokens = len(text_encoded)
    total_cost = number_tokens * cost_per_token
    return number_tokens, total_cost

In [5]:
# this function was copied from OpenAI Github account: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def input_counter(messages, model=model):
    text = [{'content': messages}] # modification made by me to deal with the inputed text
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in text:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [13]:
def input_data(response):
    date_create = response['created']
    prompted_tokens = response['usage']['prompt_tokens']
    completion_tokens = response['usage']['completion_tokens']
    msg = response['choices'][0]['message']['content'].strip()
    line = {"date_create": date_create, "prompted_tokens": prompted_tokens, "completion_tokens": completion_tokens, "msg": msg}
    df = pd.DataFrame(line, index=[0])
    return df


In [7]:
def chat(model, text, max_tokens=100):
  response = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "system", "content" : text}],
    max_tokens=max_tokens
  ) 
  return response

In [8]:
df = pd.DataFrame(columns = ["date_create", "prompted_tokens", "completion_tokens", "msg"])
df

,date_create,prompted_tokens,completion_tokens,msg


In [9]:
text = input_text()
print(text)

what is the scape velocity of mars?


In [10]:
r = chat(model, text, 1000)

In [14]:
r

<OpenAIObject chat.completion id=chatcmpl-7qACDPu7jTtNbFLPJ8xGiHOYuDcoX at 0x25be1e5a160> JSON: {
  "id": "chatcmpl-7qACDPu7jTtNbFLPJ8xGiHOYuDcoX",
  "object": "chat.completion",
  "created": 1692666605,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The escape velocity of Mars is about 5.03 kilometers per second (km/s), or about 18,108 kilometers per hour (km/h)."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 15,
    "completion_tokens": 31,
    "total_tokens": 46
  }
}

In [17]:
df2 = input_data(r)
df = df.append(df2)
df


,date_create,prompted_tokens,completion_tokens,msg
0,1692666605,15,31,The escape velocity of Mars is about 5.03 kilo...


In [21]:
df['input_cost'] = df['prompted_tokens'] * input_cost
df['output_cost'] = df['prompted_tokens'] * output_cost


In [23]:
df['total_cost'] = df['input_cost'] + df['output_cost']
df

,date_create,prompted_tokens,completion_tokens,msg,input_cost,output_cost,total_cost
0,1692666605,15,31,The escape velocity of Mars is about 5.03 kilo...,0.000023,0.00003,0.000053
0,1692666605,15,31,The escape velocity of Mars is about 5.03 kilo...,0.000023,0.00003,0.000053
